In [31]:
from sqlalchemy import create_engine, MetaData, select
import datetime as dt
import pandas as pd
import mplfinance as mpf


PG_USER = "postgres"
PG_PASSWORD = "data123"
PG_HOSTNAME = "127.0.0.1"
PG_PORT = "5432"
PG_DB_NAME = "dailypricedata"

connect = f"postgresql+psycopg2://{PG_USER}:{PG_PASSWORD}@{PG_HOSTNAME}:{PG_PORT}/{PG_DB_NAME}"
engine = create_engine(connect)

meta_data = MetaData(bind=engine)
MetaData.reflect(meta_data)
nsedata = meta_data.tables['NSEData']
query = select([
    nsedata.c.Symbol
])

stockNameList = engine.execute(query).fetchall()


In [32]:
def getStockData(engine, script):
    query = f"select * from \"{script}\""
    df = pd.read_sql_query(query, engine)
    df = pd.DataFrame(df)
    #df = df.set_index('Date')
    df["prevClose"] = df.Close.shift(1)
    df['change'] = df[['Open', 'prevClose']].pct_change(axis=1)['prevClose']
    
    return df

In [38]:
import datetime
import pendulum
startDate = pendulum.now().add(days=-30).format('YYYY-MM-DD')
granddf = pd.DataFrame()
percentageGap = 5
for script in stockNameList:
    resultdf = getStockData(engine, script[0])
    df = resultdf
    df['Date'] = pd.to_datetime(df['Date'])
    df = df[df.Date > startDate]
    df = df[df.change < -percentageGap/100]
    df['Stock'] = script[0]
    granddf = pd.concat([granddf, df])
df = granddf


In [44]:
import datetime
import pendulum
startDate = pendulum.now().add(days=-30).format('YYYY-MM-DD')
grandPositiveGapdf = pd.DataFrame()
grandNegativeGapdf = pd.DataFrame()
percentageGap = 5
for script in stockNameList:
    resultdf = getStockData(engine, script[0])
    df = resultdf
    df['Date'] = pd.to_datetime(df['Date'])
    df = df[df.Date > startDate]
    
    dfPos = df.copy()
    dfPos = dfPos[dfPos.change < -percentageGap/100]
    dfPos['Stock'] = script[0]
    
    dfNeg = df.copy()
    dfNeg = dfNeg[dfNeg.change > percentageGap/100]
    dfNeg['Stock'] = script[0]
    
    grandPositiveGapdf = pd.concat([grandPositiveGapdf, dfPos])
    grandNegativeGapdf = pd.concat([grandNegativeGapdf, dfNeg])

In [ ]:
grandNegativeGapdf

In [ ]:
grandPositiveGapdf